In [1]:
import sys
sys.path.append('../api/src')


In [2]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, AgentType, initialize_agent, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    JSONAgentOutputParser,
    ReActSingleInputOutputParser,
)
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun, tool
from langchain.tools.render import render_text_description_and_args, format_tool_to_openai_function
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder
from langchain.schema import ChatMessage, SystemMessage

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever
)
from llamp.arxiv.agents import ArxivAgent

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)

# OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
OPENAI_GPT_MODEL = "gpt-4"

/Users/oliver/code/llamp/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import re
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

mp_llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo-1106",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    max_retries=5,
    streaming=True,
	callbacks=[StreamingStdOutCallbackHandler()],
)

llm = ChatOpenAI(
    temperature=0.7,
    model="gpt-4",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    streaming=True,
	callbacks=[StreamingStdOutCallbackHandler()],
)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = [
    MPThermoExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPElasticityExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPDielectricExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPMagnetismExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPElectronicExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPPiezoelectricExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPSummaryExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPSynthesisExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPStructureRetriever(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    # ArxivAgent(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    arxiv,
    wikipedia,
]
tools += load_tools(["llm-math"], llm=mp_llm)

prompt = hub.pull("hwchase17/react-multi-input-json")
prompt.messages[0].prompt.template = re.sub(
    r"\s+", " ",
    """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
    user to clarify their queries if needed. Please note that you don't have 
    direct control over MP but through multiple assistant agents to help you. 
    You need to provide complete context in the input for them to do their job.
    """).replace("\n", " ") + prompt.messages[0].prompt.template

prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm.bind(stop=["Observation"])
    # | map_reduce_chain  # TODO: Add map-reduce after LLM
    | JSONAgentOutputParser()
)


conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

agent_kwargs = {
    "handle_parsing_errors": True,
    "extra_prompt_messages": [
        MessagesPlaceholder(variable_name="chat_history"),
        ],
    "early_stopping_method": 'generate',
}

agent_executor = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=20,
    memory=conversational_memory,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True,
)

/Users/oliver/code/llamp/venv/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [4]:
agent_executor.invoke({
    "input": "What are the bulk moduli of the following metals: Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn?",
})



> Entering new AgentExecutor chain...
To obtain the bulk moduli of the metals mentioned, I will use the MPElasticityExpert tool for each metal. I'll start with Scandium (Sc). 
Action:
```
{
  "action": "MPElasticityExpert",
  "action_input": "What is the bulk modulus of Scandium (Sc)?"
}
```To obtain the bulk moduli of the metals mentioned, I will use the MPElasticityExpert tool for each metal. I'll start with Scandium (Sc). 
Action:
```
{
  "action": "MPElasticityExpert",
  "action_input": "What is the bulk modulus of Scandium (Sc)?"
}
```

> Entering new AgentExecutor chain...


/Users/oliver/code/llamp/venv/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/Users/oliver/code/llamp/venv/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Action:
```{
  "action": "search_materials_elasticity__get",
  "action_input": {
    "formula": "Sc"
  }
}```Action:
```{
  "action": "search_materials_elasticity__get",
  "action_input": {
    "formula": "Sc"
  }
}```{"formula": "Sc"}


Retrieving ElasticityDoc documents: 100%|██████████| 5/5 [00:00<00:00, 69212.94it/s]


[{'formula_pretty': 'Sc', 'material_id': 'mp-10641', 'elastic_tensor': {'raw': [[54.47824396252669, 52.530521565025694, 52.530521565025694, -5.107025913275719e-15, -4.496403249731883e-15, -4.45476988630844e-15], [52.530521565025694, 54.47824396252669, 52.53052156502571, -5.787037515858627e-15, -5.107025913275719e-15, 9.587662765099424e-16], [52.530521565025694, 52.53052156502571, 54.47824396252669, -5.828670879282073e-15, -5.828670879282073e-15, -5.1070259132757185e-15], [-5.107025913275719e-15, -5.787037515858627e-15, -5.828670879282073e-15, 32.07361813170136, 0.0, 8.018502872736827e-31], [-4.496403249731883e-15, -5.107025913275719e-15, -5.828670879282073e-15, 0.0, 32.073618131701366, -8.902217333212434e-16], [-4.45476988630844e-15, 9.587662765099424e-16, -5.1070259132757185e-15, 8.018502872736827e-31, -8.902217333212434e-16, 32.07361813170138]], 'ieee_format': [[54.0, 53.0, 53.0, -0.0, 0.0, 0.0], [53.0, 54.0, 53.0, -0.0, -0.0, 0.0], [53.0, 53.0, 54.0, -0.0, -0.0, 0.0], [-0.0, -0.0, -

{'input': 'What are the bulk moduli of the following metals: Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn?',
 'chat_history': [],
 'output': 'The assistant should now use the MPElasticityExpert tool to find the bulk modulus of the next metal, Titanium (Ti).'}

In [5]:
print(agent_executor.memory.chat_memory.messages[1].content)

The assistant should now use the MPElasticityExpert tool to find the bulk modulus of the next metal, Titanium (Ti).


In [6]:

llm_gpt = ChatOpenAI(
    temperature=0.7,
    model="gpt-3.5-turbo-1106",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    # streaming=True
)

response = llm_gpt.invoke(agent_executor.memory.chat_memory.messages[0].content)


In [7]:
response.content

'The bulk moduli of the following metals are as follows (in GPa):\n\nSc: 122\nTi: 110\nV: 160\nCr: 160\nMn: 120\nFe: 170\nCo: 180\nNi: 180\nCu: 140\nZn: 100'

In [8]:
print(response.content)

The bulk moduli of the following metals are as follows (in GPa):

Sc: 122
Ti: 110
V: 160
Cr: 160
Mn: 120
Fe: 170
Co: 180
Ni: 180
Cu: 140
Zn: 100


In [9]:
{
  "sites": [
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [
        0,
        0,
        0
      ],
      "xyz": [
        0,
        0,
        0
      ],
      "label": "Si",
      "properties": {}
    },
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [
        0.25,
        0.25,
        0.25
      ],
      "xyz": [
        1.780000e-01,
        1.780000e-01,
        1.780000e-01
      ],
      "label": "Si",
      "properties": {}
    }
  ],
  "lattice": {
    "matrix": [
      [
        3.866002,
        0,
        2.243049
      ],
      [
        3.866002,
        3.866002,
        0
      ],
      [
        0,
        3.866002,
        2.243049
      ]
    ],
    "a": 5.431,
    "b": 5.431,
    "c": 5.431,
    "alpha": 109.471219,
    "beta": 109.471219,
    "gamma": 109.471219,
    "volume": 129.584354898
  },
  "properties": {
    "space_group": {
      "number": 227,
      "source": "spglib",
      "symbol": "Fd-3m"
    }
  }
}

{'sites': [{'species': [{'element': 'Si', 'occu': 1}],
   'abc': [0, 0, 0],
   'xyz': [0, 0, 0],
   'label': 'Si',
   'properties': {}},
  {'species': [{'element': 'Si', 'occu': 1}],
   'abc': [0.25, 0.25, 0.25],
   'xyz': [0.178, 0.178, 0.178],
   'label': 'Si',
   'properties': {}}],
 'lattice': {'matrix': [[3.866002, 0, 2.243049],
   [3.866002, 3.866002, 0],
   [0, 3.866002, 2.243049]],
  'a': 5.431,
  'b': 5.431,
  'c': 5.431,
  'alpha': 109.471219,
  'beta': 109.471219,
  'gamma': 109.471219,
  'volume': 129.584354898},
 'properties': {'space_group': {'number': 227,
   'source': 'spglib',
   'symbol': 'Fd-3m'}}}

In [10]:
from pymatgen.core import Structure

structure = Structure.from_dict(
    {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[3.333573, 0.0, 1.924639], [1.111191, 3.142924, 1.924639], [0.0, 0.0, 3.849278]], 'pbc': [True, True, True], 'a': 3.8492784033699095, 'b': 3.8492794116013456, 'c': 3.849278, 'alpha': 60.00001213094421, 'beta': 60.00000346645984, 'gamma': 60.00001097545789, 'volume': 40.32952684741405}, 'properties': {}, 'sites': [{'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.875, 0.875, 0.875], 'xyz': [3.8891685, 2.7500584999999997, 6.7362365], 'properties': {'magmom': -0.0}, 'label': 'Si'}, {'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.125, 0.125, 0.125], 'xyz': [0.5555955, 0.3928655, 0.9623195], 'properties': {'magmom': -0.0}, 'label': 'Si'}, {'species': [{'element': 'Li', 'occu': 1}], 'abc': [0.5, 0.5, 0.5], 'xyz': [2.222382, 1.571462, 1.924639], 'properties': {}, 'label': 'Li'}]}
)

structure.to(filename="../experiments/05-crystal-Si-Li-interstitial.cif", fmt="cif")
structure.to_ase_atoms().write("../experiments/05-crystal-Si-Li-interstitial.xyz")

/Users/oliver/code/llamp/venv/lib/python3.11/site-packages/pymatgen/core/structure.py:1020: UserWarning: Not all sites have property magmom. Missing values are set to None.
  warnings.warn(f"Not all sites have property {key}. Missing values are set to None.")


PackageNotFoundError: No package metadata was found for AseAtomsAdaptor requires the ASE package. Use `pip install ase`

In [ ]:
structure = Structure.from_dict(
    {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[3.700773, -0.991615, 2.3e-05], [-0.991617, 3.700765, -2.4e-05], [3e-05, -3.2e-05, 6.332622]], 'pbc': [True, True, True], 'a': 3.8313210654137304, 'b': 3.83131385564926, 'c': 6.332622000151911, 'alpha': 90.00070882372823, 'beta': 89.99931892714707, 'gamma': 119.99994034554126, 'volume': 80.50275361604729}, 'properties': {}, 'sites': [{'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.666683, 0.333317, 0.500012], 'xyz': [2.13673464273, 0.572419024076, 3.166394325565], 'properties': {'magmom': 0.0}, 'label': 'Si'}, {'species': [{'element': 'Si', 'occu': 1}], 'abc': [1.5e-05, 0.999985, 9e-06], 'xyz': [-0.99154661388, 3.7006946140120003, 3.2994303e-05], 'properties': {'magmom': 0.0}, 'label': 'Si'}, {'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.666683, 0.333318, 0.873993], 'xyz': [2.136744870543, 0.5724107574489999, 5.534674633723], 'properties': {'magmom': 0.0}, 'label': 'Si'}, {'species': [{'element': 'Si', 'occu': 1}], 'abc': [2e-05, 0.999981, 0.373986], 'xyz': [-0.991512924237, 3.700662885613, 2.368287972208], 'properties': {'magmom': 0.0}, 'label': 'Si'}]}
)

structure.to(filename="../experiments/05-crystal-Si.cif", fmt="cif")
structure.to_ase_atoms().write("../experiments/05-crystal-Si.xyz")

In [ ]:

structure = Structure.from_dict(
    {
  "sites": [
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [0, 0, 0],
      "xyz": [0, 0, 0],
      "label": "Si",
      "properties": {}
    },
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [0.25, 0.25, 0.25],
      "xyz": [0.178, 0.178, 0.178],
      "label": "Si",
      "properties": {}
    },
    {
      "species": [
        {
          "element": "Li",
          "occu": 1
        }
      ],
      "abc": [0.5, 0.5, 0.5],
      "xyz": [0.5, 0.5, 0.5],
      "label": "Li",
      "properties": {}
    }
  ],
  "lattice": {
    "matrix": [
      [3.866002, 0, 2.243049],
      [3.866002, 3.866002, 0],
      [0, 3.866002, 2.243049]
    ],
    "a": 5.431,
    "b": 5.431,
    "c": 5.431,
    "alpha": 109.471219,
    "beta": 109.471219,
    "gamma": 109.471219,
    "volume": 129.584354898
  },
  "properties": {
    "space_group": {
      "number": 227,
      "source": "spglib",
      "symbol": "Fd-3m"
    }
  }
}
)

structure.to(filename="../experiments/05-crystal-Si-Li-interstitial-gpt3.5.cif", fmt="cif")
structure.to_ase_atoms().write("../experiments/05-crystal-Si-Li-interstitial-gpt3.5.xyz")